In [1]:
import os
import torch
import sys

sys.path.append('I:/mmdeploy/build/bin/Release')
os.add_dll_directory('D:/opencv/build/x64/vc16/bin')
os.add_dll_directory('D:/onnxruntime-win-x64-1.8.1/lib')

<AddedDllDirectory('D:/onnxruntime-win-x64-1.8.1/lib')>

In [2]:
from mmdeploy_python import Detector, PoseDetector

# hand pose

In [3]:
detector = Detector('hand2_yolov8n', 'cpu', 0)
pose_detector = PoseDetector('hand', 'cpu', 0)

In [ ]:
import cv2

def draw_labels(frame, bbox):
        [left, top, right, bottom] = bbox[0:4].astype(int)
        # coco数据集标签对应的颜色
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 1)

In [53]:
img = cv2.imread('test.jpg')

bboxes, label, _ = detector(img)

[left, top, right, bottom] = bboxes[0][0:4].astype(int)


result = pose_detector(img, bboxes[..., :4])

scores = result[..., 2]
keypoints = (result[..., :2]).astype(int)

print(keypoints)

skeleton = [(1, 0), (1, 2), (2, 3), (3, 4), (5, 0), 
            (5, 6), (6, 7), (7, 8), (12, 0), 
            (9, 10), (10, 11), (11, 12), (16, 0), 
            (13, 14), (14, 15), (15, 16), (20, 0), 
            (17, 18), (18, 19), (19, 20)]

for kpts, score in zip(keypoints, scores):

  for (u,v) in skeleton:
    cv2.line(img, (kpts[u][0], kpts[u][1]), (kpts[v][0], kpts[v][1]), (0, 255, 0), 1, cv2.LINE_AA)

  for kpt in range(len(kpts)):
    if kpt == 0:
      cv2.circle(img, kpts[kpt], 1, (255, 0, 0), 2, cv2.LINE_AA)
    else:
      cv2.circle(img, kpts[kpt], 1, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imwrite('pose.jpg', img)

[[[407 146]
  [400 237]
  [398 218]
  [410 194]
  [405 168]
  [264 245]
  [297 223]
  [324 206]
  [355 180]
  [235 237]
  [266 211]
  [297 187]
  [333 163]
  [228 211]
  [259 192]
  [292 170]
  [324 151]
  [232 175]
  [261 166]
  [280 158]
  [319 146]]]


True

# whole body

In [4]:
detector = Detector('yolov8', 'cpu', 0)
pose_detector = PoseDetector('wholebody', 'cpu', 0)

In [19]:
def visualize(frame, keypoints, thr=0.0):
        skeleton = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12), (5, 11),
                    (6, 12), (5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (1, 2),
                    (0, 1), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6)]
        palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
                (255, 153, 255), (153, 204, 255), (255, 102, 255),
                (255, 51, 255), (102, 178, 255),
                (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
                (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
                (0, 0, 255), (255, 0, 0), (255, 255, 255)]
        link_color = [
            0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
        ]
        point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]

        scores = keypoints[..., 2]
        keypoints = keypoints[..., :2].astype(int)

        for kpts, score in zip(keypoints, scores):
            show = [0] * len(kpts)
            for (u, v), color in zip(skeleton, link_color):
                if score[u] > thr and score[v] > thr:
                    cv2.line(frame, kpts[u], tuple(kpts[v]), palette[color], 1,
                            cv2.LINE_AA)
                    show[u] = show[v] = 1
            for kpt, show, color in zip(kpts, show, point_color):
                if show:
                    cv2.circle(frame, kpt, 1, palette[color], 2, cv2.LINE_AA)

def visualize_hand(frame, keypoints, thr=0.0):
        skeleton = [(0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (5, 6), (6, 7),
                    (7, 8), (0, 9), (9, 10), (10, 11), (11, 12), (0, 13),
                    (13, 14), (14, 15), (15, 16), (0, 17), (17, 18), (18, 19),
                    (19, 20)]
        palette = [(255, 128, 0), (255, 153, 51), (255, 178, 102), (230, 230, 0),
                (255, 153, 255), (153, 204, 255), (255, 102, 255),
                (255, 51, 255), (102, 178, 255),
                (51, 153, 255), (255, 153, 153), (255, 102, 102), (255, 51, 51),
                (153, 255, 153), (102, 255, 102), (51, 255, 51), (0, 255, 0),
                (0, 0, 255), (255, 0, 0), (255, 255, 255)]
        link_color = [
            0, 0, 0, 0, 7, 7, 7, 7, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
        ]
        point_color = [16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0]


        scores = keypoints[..., 2]
        keypoints = keypoints[..., :2].astype(int)

        for kpts, score in zip(keypoints, scores):
            show = [0] * len(kpts)
            for (u, v), color in zip(skeleton, link_color):
                if score[u] > thr and score[v] > thr:
                    cv2.line(frame, kpts[u], tuple(kpts[v]), palette[color], 1,
                            cv2.LINE_AA)
                    show[u] = show[v] = 1
            for kpt, show, color in zip(kpts, show, point_color):
                if show:
                    cv2.circle(frame, kpt, 1, palette[color], 2, cv2.LINE_AA)

In [20]:
import cv2

img = cv2.imread('whole_body3.jpg')

bboxes, label, _ = detector(img)

result = pose_detector(img, bboxes[1, :4])
scores = result[..., 2]
keypoints = (result[..., :2]).astype(int)

for  kpts, score in zip(keypoints, scores):
  for kpt in range(len(kpts)):
    if kpt >=23 and kpt <= 95:
      cv2.circle(img, kpts[kpt], 1, (255, 0, 0), 8, cv2.LINE_AA)
        # elif kpt >= 0 and kpt <= 16:
        #   cv2.circle(img, kpts[kpt], 1, (0, 255, 0), 8, cv2.LINE_AA)
        # elif kpt >= 91 and kpt <= 111:
        #   cv2.circle(img, kpts[kpt], 1, (0, 0, 255), 8, cv2.LINE_AA)
        # elif kpt >= 112 and kpt <= 132:
        #   cv2.circle(img, kpts[kpt], 1, (255, 0, 255), 8, cv2.LINE_AA)  

visualize(img, keypoints=result[:,:17,:])
visualize_hand(img, keypoints=result[:,91:112,:])
visualize_hand(img, keypoints=result[:,112:133,:])
cv2.imwrite('whole_body_pose3.jpg', img)

True